In [16]:
import time
import datetime
import random
from proton_driver import client
import plotly.graph_objs as go
from IPython.display import clear_output, display

In [17]:
c = client.Client(host='192.168.1.10', port=8463)

In [18]:
# create a random stream if not exist
c.execute("""
        CREATE STREAM IF NOT EXISTS price_data(
            TIMESTAMP datetime,
            price float
        )
""")

[]

In [19]:
base_price = 100
current_price = base_price
volatility = 0.02

def generate_price():
    global current_price
    change_percent = random.uniform(-volatility, volatility)
    current_price = current_price * (1 + change_percent)
    noise = random.uniform(-0.5, 0.5)

    if current_price < 10:
        current_price = 10 + random.uniform(-0.5,0.5)
    return round(current_price, 2)

In [20]:
def write_data():
    counter = 0
    try:
        while True:
            timestamp = datetime.datetime.now()
            price = generate_price()
            c.execute( f"""
                INSERT INTO price_data (TIMESTAMP, price)
                VALUES ('{timestamp.strftime("%Y-%m-%d %H:%M:%S")}', {price})
        """)
            print(f"Inserted: {timestamp}, Price: {price}")
            counter += 1
            time.sleep(1)  # Sleep for 1 second between inserts
    
    except KeyboardInterrupt:
        print(f"Inserted {counter} records before stopping.")
    

In [ ]:
write_data()

Inserted: 2025-06-06 14:16:19.250024, Price: 100.29
Inserted: 2025-06-06 14:16:20.258800, Price: 100.46
Inserted: 2025-06-06 14:16:21.274712, Price: 99.71
Inserted: 2025-06-06 14:16:22.291961, Price: 101.32
Inserted: 2025-06-06 14:16:23.306914, Price: 102.47
Inserted: 2025-06-06 14:16:24.319756, Price: 102.72
Inserted: 2025-06-06 14:16:25.328379, Price: 101.94
Inserted: 2025-06-06 14:16:26.339730, Price: 102.92
Inserted: 2025-06-06 14:16:27.352314, Price: 102.87
Inserted: 2025-06-06 14:16:28.367878, Price: 101.03
Inserted: 2025-06-06 14:16:29.379450, Price: 101.66
Inserted: 2025-06-06 14:16:30.396830, Price: 102.93
Inserted: 2025-06-06 14:16:31.408625, Price: 104.41
Inserted: 2025-06-06 14:16:32.425216, Price: 102.85
Inserted: 2025-06-06 14:16:33.435500, Price: 101.34
Inserted: 2025-06-06 14:16:34.449802, Price: 103.28
Inserted: 2025-06-06 14:16:35.464916, Price: 104.74
Inserted: 2025-06-06 14:16:36.476577, Price: 106.11
Inserted: 2025-06-06 14:16:37.489006, Price: 105.99
Inserted: 202

In [ ]:
def test():
    rows = c.execute_iter(
        "SELECT TIMESTAMP, price FROM price_data ORDER BY TIMESTAMP DESC LIMIT 10"
    )
    for row in rows:
        print(row)